## Imports and data loading

In [ ]:
scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)

```{note}
You can modify `save_dir` below to change where the data files for this tutorial are saved.
```

In [ ]:
sc.set_figure_params(figsize=(6, 6), frameon=False)
sns.set_theme()
torch.set_float32_matmul_precision("high")
save_dir = tempfile.TemporaryDirectory()

%config InlineBackend.print_figure_kwargs={"facecolor": "w"}
%config InlineBackend.figure_format="retina"

### Using your own data

If you want to analyze your own data, there are multiple ways to upload the count matrix \(in h5 or mtx format\):
* It can be upload directly to the colab session storage, use the `Upload to session storage` button on the upper left corner in the `Files` tab.
* You can also upload the data to Google Drive, and mount it using the Mount Drive button in the Files tab (for Colab users only).
* Download directly from public available data source using command line tools such as curl. This will work not only for Colab.

In this example we are downloading h5 format of single-cell multiomic data generated by Proteintech Genomics directly into the Google Colab session storage. The data is from human resting PBMCs stained with the MultiPro® Human Discovery Panel (HDP) followed by processing using 10x Genomics Flex chemistry with Feature Barcoding Technology.

In [ ]:
!curl -LO https://ptgngsdata.s3.us-west-2.amazonaws.com/counts/E44_1_restPBMC_DCpos_filtered_feature_bc_matrix.h5

We now run a preprocessing workflow:

mdata1 = muon.read_10x_h5("E44_1_restPBMC_DCpos_filtered_feature_bc_matrix.h5")
mdata1.mod["rna"].var_names_make_unique()

In [ ]:
mdata1.mod["rna"].layers["counts"] = mdata1.mod["rna"].X.copy()
sc.pp.normalize_total(mdata1.mod["rna"])
sc.pp.log1p(mdata1.mod["rna"])

sc.pp.highly_variable_genes(
    mdata1.mod["rna"],
    n_top_genes=4000,
    flavor="seurat_v3",
    layer="counts",
)
# Place subsetted counts in a new modality
mdata1.mod["rna_subset"] = mdata1.mod["rna"][:, mdata1.mod["rna"].var["highly_variable"]].copy()

Because of the filtering process we will re create the mdata here

In [ ]:
mdata1 = md.MuData(mdata1.mod)

# we need to work with dense and not sparse matrices:
mdata1["prot"].X = mdata1["prot"].X.toarray()
mdata1["rna_subset"].X = mdata1["rna_subset"].X.toarray()
mdata1.mod["rna_subset"].layers["counts"] = mdata1.mod["rna_subset"].layers["counts"].toarray()

In [ ]:
mdata1.update()

In [ ]:
mdata1